<a href="https://colab.research.google.com/github/manguetownson/Classificacao_de_dispositivos_python_cnn/blob/main/dispositives_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import glob
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import seaborn as sns
import cv2
import os
import imageio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, RepeatedStratifiedKFold
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet import preprocess_input
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
!pip install --upgrade gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [ ]:
!gdown 1voZSk_amRVxnJ30FEL2GiSooF_cnGZtw

Downloading...
From (uriginal): https://drive.google.com/uc?id=1voZSk_amRVxnJ30FEL2GiSooF_cnGZtw
From (redirected): https://drive.google.com/uc?id=1voZSk_amRVxnJ30FEL2GiSooF_cnGZtw&confirm=t&uuid=1070807e-17ef-4542-9613-a9328b03b4bb
To: /content/dataset_dispositivos.zip
100% 1.28G/1.28G [00:25<00:00, 49.2MB/s]


In [ ]:
!unzip -q 'dataset_dispositivos.zip'

In [ ]:
batch_size = 32

data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    dtype='float32'
)

train_generator = data_generator.flow_from_directory(
    '/content/dataset_dispositivos/',  #em caso de teste, alterar o nome do arquivo em /content de "dataset dispositivos" para dataset_dispositivos"
    target_size=(224, 224),            #também alterar a classe "dispositivos_móveis" para "dispositivos_moveis (para que não haja erros, pois cometi esse deslize na criação da base)
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    '/content/dataset_dispositivos/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 2182 images belonging to 5 classes.
Found 544 images belonging to 5 classes.


In [ ]:
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (4, 4), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))  # Ajuste para corresponder ao número real de classes (5)
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(
    train_generator,
    epochs=15,
    validation_data=train_generator
    )

Epoch 1/50
69/69 [==============================] - 416s 6s/step - loss: 1.5253 - accuracy: 0.3034 - val_loss: 1.3964 - val_accuracy: 0.4189
Epoch 2/50
69/69 [==============================] - 395s 6s/step - loss: 1.3269 - accuracy: 0.3918 - val_loss: 1.2178 - val_accuracy: 0.4028
Epoch 3/50
69/69 [==============================] - 400s 6s/step - loss: 1.2585 - accuracy: 0.4372 - val_loss: 1.0843 - val_accuracy: 0.5229
Epoch 4/50
69/69 [==============================] - 400s 6s/step - loss: 1.1523 - accuracy: 0.4945 - val_loss: 1.0995 - val_accuracy: 0.5101
Epoch 5/50
69/69 [==============================] - 403s 6s/step - loss: 1.1005 - accuracy: 0.5211 - val_loss: 0.9014 - val_accuracy: 0.6434
Epoch 6/50
69/69 [==============================] - 406s 6s/step - loss: 1.0468 - accuracy: 0.5761 - val_loss: 0.8566 - val_accuracy: 0.6810
Epoch 7/50
69/69 [==============================] - 395s 6s/step - loss: 0.9599 - accuracy: 0.6159 - val_loss: 0.8818 - val_accuracy: 0.7163
Epoch 8/50
69

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

# Definir o caminho das pastas de treinamento e teste
caminho_treinamento = '/content/dataset_dispositivos'


# Definir dimensões das imagens
largura, altura = 224, 224

# Criar um objeto ImageDataGenerator para pré-processamento dos dados de teste
test_datagen = ImageDataGenerator(rescale=1./255)

# Carregar os dados de teste
dados_teste = test_datagen.flow_from_directory(
    caminho_treinamento,
    target_size=(largura, altura),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# Fazer previsões nos dados de teste
previsoes = model.predict_generator(dados_teste, steps=dados_teste.samples)

# Obter as classes preditas
classes_preditas = np.argmax(previsoes, axis=1)

# Obter as classes verdadeiras
classes_verdadeiras = dados_teste.classes

# Calcular a acurácia
acuracia = np.mean(classes_preditas == classes_verdadeiras)
print("Acurácia do modelo nos dados de teste:", acuracia)



Found 221 images belonging to 3 classes.
Acurácia do modelo nos dados de teste: 0.8823529411764706


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications import VGG16
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Create an instance of the VGG16 model with pre-trained weights
vgg = VGG16(include_top=True,
            weights="imagenet",
            classifier_activation="softmax",
            #input_tensor=(None),
            #input_shape=(224, 224, 3),
            #pooling=(2, 2),
            #classes=1000,
           )

# Freeze the convolutional layers in the base model so they are not updated during training
for layer in vgg.layers:
    layer.trainable = False

# Add your own classification layers on top of the base model
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))


553467096/553467096 [==============================] - 24s 0us/step


In [ ]:
# Compile the model with a lower learning rate and categorical hinge loss
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



# Fit the model using the augmented data generator
history_vgg16 = model.fit(train_generator,
                    epochs=15,
                    validation_data = train_generator,
                    )

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

# Definir o caminho das pastas de treinamento e teste
caminho_treinamento = '/content/treinamento'


# Definir dimensões das imagens
largura, altura = 224, 224

# Criar um objeto ImageDataGenerator para pré-processamento dos dados de teste
test_datagen = ImageDataGenerator(rescale=1./255)

# Carregar os dados de teste
dados_teste = test_datagen.flow_from_directory(
    caminho_teste,
    target_size=(largura, altura),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# Fazer previsões nos dados de teste
previsoes = model.predict_generator(dados_teste, steps=dados_teste.samples)

# Obter as classes preditas
classes_preditas = np.argmax(previsoes, axis=1)

# Obter as classes verdadeiras
classes_verdadeiras = dados_teste.classes

# Calcular a acurácia
acuracia = np.mean(classes_preditas == classes_verdadeiras)
print("Acurácia do modelo nos dados de teste:", acuracia)



In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications import VGG19

# Create an instance of the VGG16 model with pre-trained weights
vgg1 = VGG19(include_top=True,
            weights="imagenet",
            classifier_activation="softmax",
            #input_tensor=(None),
            #input_shape=(224, 224, 3),
            #pooling=(2, 2),
            #classes=1000,
           )

# Freeze the convolutional layers in the base model so they are not updated during training
for layer in vgg1.layers:
    layer.trainable = False

# Add your own classification layers on top of the base model
model = Sequential()
model.add(vgg1)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))


In [ ]:
# Compile the model with a lower learning rate and categorical hinge loss
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



# Fit the model using the augmented data generator
history_vgg19 = model.fit(train_generator,
                    epochs=15,
                    validation_data = train_generator,
                    )